## Scenario

You worked on creating market segmentation groups using kmeans on customer shopping records, but you're not sure kmeans was the best method. By the end of this lesson, you want to find the appropriate number of groups and their descriptions.

# Hierchical Clustering 
Agenda today:
- Hierarchical Clustering 
    -  Introduction
    -  How the algorithm is constructed?
    -  Implementing hierarchical algorithm

Goals today:
- Understand how hierarchical clustering finds clusterings in observations
- Compare and contrast hierchical clustering and k-means
- Build and interpret a dendrogram
***

# Introduction 

### Supervised vs. Unsupervised Machine Learning 
Recall from our previous lesson in supervised learning, where we use a number of features to predict a label. In unsupervised ML problem, we are not predicting and labels, thus we do not have a ground truth to compare our models to. In other words, we are doing the best we can to group individual observations together without having a gold standard to evaluate them. 

### Intuition
Clustering is a family of techniques for identifying clusters in a dataset. The goal of clustering is to group the most similar observations together into a cluster. In Hierchical Clustering, we prioritize **similarity** between individual observations.


### PCA vs. Clustering
Both PCA and clustering are unsupervised algorithms. The difference between them are, according to ISLR:
- PCA looks to find low-dimensional representation of the observations that explain a good fraction of the variance 
- Clustering looks to find homogenous subgroups among the observations
***

# Hierarchical Agglomerative Clustering
Recall K-means clustering where the goal is to assign individual observations to pre-specified number of clusters according to Euclidean distance between the centroid and the observation, Hierarchical clustering sets out to group the most similar two observations together from a bottom-up level. We end up with a tree-like diagram named **dendrogram**, which allows us to view the clusterings obtained for each possible number of clusters, from 1 to n. It is up to our discretion as data scientists to decide how many clusters we want. 

![dendo](img/dendogram.png)

***
One disadvantage of K-means clustering is that we have to specify the number of clusters beforehand. The type of hierchical clustering we will learn today is **agglomerative**, or **bottom up**, such that we do not have to specify the number of clusters beforehand. We will now dive into the details of hierchical clustering.
***

### How does the algorithm work
Initially, every observation is its own cluster. As we move up the leaf of the dendrogram, the most similar pair of observation fuse together, and the next most similar group of leafs fuse together etc. until everything fuse together into a big cluster. Where to cut off branching that fuse together the tree is up to our discretion. 

![dendo2](img/400_Basic_Dendrogram.png)

## Aglomerative example

In [ ]:
from hier_example import *

In [ ]:
plot_agglomerative_algorithm()

### Types of hierarchical aggplomerative clustering 
- Single Linkage 
    -  Minimum pair-wise distance: for any two clusters, take one observation from each and determine their distance. Do this over and over, until you have identified the overall minimum pair-wise distance. 
- Complete Linkage
    -  Nearest may be defined as the furthest (or maximum) distance between two clusters. That is, all possible pairwise distances between elements (one from cluster A and one from B) are evaluated and the largest value is used as the distance between clusters A & B. This is sometimes called complete linkage and is also called furthest neighbor.
- Average Linkage
    - The distance between clusters is defined as the average distance between the average values of each of the data points in the clusters. 
- Ward Linkage
    -  Ward method finds the pair of clusters that leads to minimum increase in total within-cluster variance after merging at each step.

I really like how **[this article](https://towardsdatascience.com/understanding-the-concept-of-hierarchical-clustering-technique-c6e8243758ec)** describes the pros and cons of each approach.

### How well does the dendogram fit the data?
cophenetic correlation coefficient $c$ is given by [ref](https://www.mathworks.com/help/stats/index.html?/access/helpdesk/help/toolbox/stats/cophenet.html=)

![c-coef](img/cophenet.png)


$x(i, j) = | Xi − Xj |$, the ordinary Euclidean distance between the $i$th and $j$th observations.<br>
$t(i, j)$ = the dendrogrammatic distance between the model points $Ti$ and $Tj$. This distance is the height of the node at which these two points are first joined together.<br>

Then, letting ${\bar {x}}$ be the average of the $x(i, j)$, and letting ${\bar {t}}$ be the average of the $t(i, j)$, the cophenetic correlation coefficient $c$ is given by[4]

## Seeing it in action

**[This post here](https://www.analyticsvidhya.com/blog/2019/05/beginners-guide-hierarchical-clustering/)** walks through cluster assignment _step_ by _step_ if the demo would be helpful.

Meanwhile, we can do it in _**scipy**_ and _**sklearn**_

### Hierarchical clustering with `scipy`

In [ ]:
# lets generate some data and look at an example of hierarchical agglomerative clustering 
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
# generate two clusters: a with 100 points, b with 50:
np.random.seed(1000)  
a = np.random.multivariate_normal([10, 0], [[3, 1], [1, 4]], size=[50,])
b = np.random.multivariate_normal([0, 20], [[3, 1], [1, 4]], size=[50,])
X = np.concatenate((a, b),)
print(X.shape)  # 150 samples with 2 dimensions
plt.scatter(X[:,0], X[:,1])
plt.title("Sample data for clustering demo")

In [ ]:
# construct dendrogram in scipy
from scipy.cluster.hierarchy import dendrogram, linkage
Z = linkage(X, 'single')

In [ ]:
from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist

c, coph_dists = cophenet(Z, pdist(X))
c

In [ ]:
# calculate and construct the dendrogram 
# calculate full dendrogram
plt.figure(figsize=(25, 10))
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('sample index')
plt.ylabel('distance')
dendrogram(
    Z,
    leaf_rotation=90.,  # rotates the x axis labels
    leaf_font_size=8.,  # font size for the x axis labels
)
plt.show()

In [ ]:
# trimming and truncating the dendrogram 
plt.title('Hierarchical Clustering Dendrogram (truncated)')
plt.xlabel('sample index')
plt.ylabel('distance')
dendrogram(
    Z,
    truncate_mode='lastp',  # show only the last p merged clusters
    p=12,  # show only the last p merged clusters
    show_leaf_counts=False,  # otherwise numbers in brackets are counts
    leaf_rotation=90.,
    leaf_font_size=12.,
    show_contracted=True,  # to get a distribution impression in truncated branches
)
plt.show()

# from documentation of "lastp"
# The last p non-singleton formed in the linkage are the only non-leaf nodes in the linkage; 
# they correspond to rows Z[n-p-2:end] in Z. All other non-singleton clusters are contracted into leaf nodes.

### Hierarchical clustering with `sklearn` on Iris (because it's there)

**[Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html)** for AgglomerativeClustering in `sklearn`


**[A great example of using manhattan distance](https://scikit-learn.org/stable/auto_examples/cluster/plot_agglomerative_clustering_metrics.html#sphx-glr-auto-examples-cluster-plot-agglomerative-clustering-metrics-py)** with agglomerative clustering in `sklearn`.

In [ ]:
# we can also use the scikitlearn module hierarchical clustering to perform the same task 
from sklearn.datasets import make_blobs
from sklearn.datasets import make_moons
from sklearn.cluster import AgglomerativeClustering
from sklearn.neighbors import KernelDensity
np.random.seed(2000)

In [ ]:
# try clustering on the iris dataset 
from sklearn.datasets import load_iris
iris = load_iris()
# in this case, we won't be working with predicting labels, so we will only use the features (X)
X_iris = iris.data
y_iris = iris.target

In [ ]:
plt.scatter(X_iris[:,0],X_iris[:,2]) #c = y_iris)

In [ ]:
iris_cluster = AgglomerativeClustering(n_clusters=3)
iris_cluster
pred_iris_clust = iris_cluster.fit_predict(X_iris)
plt.scatter(X_iris[:, 0], X_iris[:, 2], c = pred_iris_clust, s = 10);

In [ ]:
# compare it to the actual truth 
plt.scatter(X_iris[:,0],X_iris[:,2], c = y_iris)

#### Evaluate

To evaluate you might try different numbers of clusters and compare their silhouette score as you did w kmeans.

In [ ]:
# evaluation - silhouette score 
from sklearn.metrics import silhouette_score
silhouette_score(X_iris, pred_iris_clust)

### Evaluating number of clusters / Cut points
For hierarchical agglomerative clustering, or clustering in general, it is generally difficult to truly evaluate the results. Therefore, it is up you, the data scientists, to decide.

**[Standford has a good explaination on page 380](https://nlp.stanford.edu/IR-book/pdf/17hier.pdf)** of your options for picking the cut-off. 

When we are viewing dendrograms for hierarchical agglomerative, we can visually examine where the natural cutoff is, despite it not sounding exactly statistical, or scientific. We might want to interpret the clusters and assign meanings to them depending on domain-specific knowledge and shape of dendrogram. However, we can evaluate the quality of our clusters using measurements such as Sihouette score discussed in the k-means lectures. 



## Advantages & Disadvantages of hierarchical clustering

#### Advantages
- Intuitive and easy to implement
- More informative than k-means because it takes individual relationship into consideration
- Allows us to look at dendrogram and decide number of clusters

#### Disadvantages
- Very sensitive to outliers
- Cannot undo the previous merge, which might lead to problems later on 


### Further reading

- [from MIT on just hierarchical](http://web.mit.edu/6.S097/www/resources/Hierarchical.pdf)
- [from MIT comparing clustering methods](http://www.mit.edu/~9.54/fall14/slides/Class13.pdf)
- [fun CMU slides on clustering](http://www.cs.cmu.edu/afs/andrew/course/15/381-f08/www/lectures/clustering.pdf)

### Find those clusters!!! 

In [1]:
import numpy as np
import pandas as pd

In [2]:
shop = pd.read_excel('Online Retail.xlsx')

In [7]:
shop.dropna(inplace= True)

In [8]:
shop.isna().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [10]:
df = pd.DataFrame(shop['CustomerID'])

In [11]:
df['CustomerID'].astype('int')

0         17850
1         17850
2         17850
3         17850
4         17850
5         17850
6         17850
7         17850
8         17850
9         13047
10        13047
11        13047
12        13047
13        13047
14        13047
15        13047
16        13047
17        13047
18        13047
19        13047
20        13047
21        13047
22        13047
23        13047
24        13047
25        13047
26        12583
27        12583
28        12583
29        12583
          ...  
541879    15804
541880    15804
541881    15804
541882    15804
541883    15804
541884    15804
541885    15804
541886    15804
541887    15804
541888    15804
541889    15804
541890    13113
541891    13113
541892    13113
541893    13113
541894    12680
541895    12680
541896    12680
541897    12680
541898    12680
541899    12680
541900    12680
541901    12680
541902    12680
541903    12680
541904    12680
541905    12680
541906    12680
541907    12680
541908    12680
Name: CustomerID, Length

In [24]:
shop['CustomerID'] = shop['CustomerID'].astype('int')

In [17]:
shop['Revenue'] = np.round(shop['Quantity']*shop['UnitPrice'], 2)

In [25]:
shop['CustomerID']

0         17850
1         17850
2         17850
3         17850
4         17850
5         17850
6         17850
7         17850
8         17850
9         13047
10        13047
11        13047
12        13047
13        13047
14        13047
15        13047
16        13047
17        13047
18        13047
19        13047
20        13047
21        13047
22        13047
23        13047
24        13047
25        13047
26        12583
27        12583
28        12583
29        12583
          ...  
541879    15804
541880    15804
541881    15804
541882    15804
541883    15804
541884    15804
541885    15804
541886    15804
541887    15804
541888    15804
541889    15804
541890    13113
541891    13113
541892    13113
541893    13113
541894    12680
541895    12680
541896    12680
541897    12680
541898    12680
541899    12680
541900    12680
541901    12680
541902    12680
541903    12680
541904    12680
541905    12680
541906    12680
541907    12680
541908    12680
Name: CustomerID, Length

In [27]:
shop['Order_Count'] = 1

In [48]:
shop

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Order_Count,Revenue,Purchase_Avg
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,1,15.30,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,1,20.34,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,1,22.00,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,1,20.34,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,1,20.34,20.34
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01 08:26:00,7.65,17850,United Kingdom,1,15.30,15.30
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,2010-12-01 08:26:00,4.25,17850,United Kingdom,1,25.50,25.50
7,536366,22633,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.85,17850,United Kingdom,1,11.10,11.10
8,536366,22632,HAND WARMER RED POLKA DOT,6,2010-12-01 08:28:00,1.85,17850,United Kingdom,1,11.10,11.10
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047,United Kingdom,1,54.08,54.08


In [46]:
shop.groupby('CustomerID').sum()

,Quantity,UnitPrice,Order_Count,Revenue,Purchase_Avg
CustomerID,,,,,
12346,0,2.08,2,0.000000e+00,0.000000e+00
12347,2458,481.21,182,4.310000e+03,4.310000e+03
12348,2341,178.71,31,1.797240e+03,1.797240e+03
12349,631,605.10,73,1.757550e+03,1.757550e+03
12350,197,65.30,17,3.344000e+02,3.344000e+02
12352,470,2211.10,95,1.545410e+03,1.545410e+03
12353,20,24.30,4,8.900000e+01,8.900000e+01
12354,530,261.22,58,1.079400e+03,1.079400e+03
12355,240,54.65,13,4.594000e+02,4.594000e+02


In [ ]:
Neg_quantity = list(shop['Quantity'] < 0)
Neg_quant = []
for i in range(0, len(shop)):
    if Neg_quantity[i] == True:
        Neg_quant.append(i)
    else:
        pass
len(Neg_quant)

In [ ]:
##There are no negative prices

In [ ]:
Neg_pricing = list(shop['UnitPrice'] < 0)
Neg_price = []
for i in range(0, len(shop)):
    if Neg_pricing[i] == True:
        Neg_price.append(i)
    else:
        pass
len(Neg_price)

In [ ]:
sum(shop['Quantity'] < 0)

In [ ]:
shop['Revenue'] = np.round(shop['Quantity']*shop['UnitPrice'], 2)

In [ ]:
##C in invoice number represents a return

In [ ]:
shop.groupby('InvoiceNo').sum().sort_values(by=['CustomerID'])

In [ ]:
shop.

In [ ]:
shop['InvoiceDate'] = pd.to_datetime(shop['InvoiceDate'])

In [ ]:
type(shop['InvoiceDate'][0])

In [ ]:
shop['CustomerID'].head().append(shop['CustomerID'].tail())

In [ ]:
shop.dtypes

In [ ]:
shop.isna().sum()

In [ ]:
shop.dropna(inplace= True)

In [ ]:
shop.isna().sum()

In [ ]:
shop.iloc[:,:]

In [ ]:
#standardize the data to normal distribution
from sklearn import preprocessing
dataset1_standardized = preprocessing.scale(shop)
dataset1_standardized = pd.DataFrame(shop)